In [ ]:
# Setup
%load_ext autoreload
%autoreload 2
%matplotlib notebook

# Lesson 8: Fitting Copulas to Data

This notebook covers the practical aspects of fitting copulas to observed data. We will learn how to transform data to pseudo-observations, use rank-based dependence measures like Kendall's tau, and estimate copula parameters.

## Learning Objectives
- Understand the pseudo-observation transformation
- Learn the relationship between Kendall's tau and copula parameters
- Implement parameter estimation for common copula families

In [ ]:
# Import required libraries
import matplotlib.pyplot as plt
from IPython import display
import seaborn as sns
sns.set_theme(style="whitegrid")

import numpy as np
import math
from scipy.stats import norm, \
    beta, cauchy, expon, rayleigh, uniform, multivariate_t, t, \
    rankdata, kendalltau, pearsonr
from scipy.stats.mstats import spearmanr
import scipy.integrate as integrate
import pandas as pd
from scipy.optimize import brentq, fsolve

## Pseudo-Observations

To fit a copula to data, we first need to transform the data to the unit cube. This is done using **pseudo-observations** (also called empirical copula data or probability integral transforms).

The idea is to replace each observation with its normalized rank. This removes the marginal distributions and leaves only the dependence structure.

### Example: Original Data

| x      | y |
| ----------- | ----------- |
| 0.6      | 0.8       |
| 0.2   | 0.4        |
| 1.2   | 0.5        |
| 0.1   | 0.2        |

### Rank Observations

After ranking (smallest = 1, largest = n):

| x      | y |
| ----------- | ----------- |
| 3   | 4    |
| 2   | 2    |
| 4   | 3    |
| 1   | 1    |

The pseudo-observations are then $U_i = R_i / (n+1)$, where $R_i$ is the rank. We divide by $n+1$ instead of $n$ to avoid boundary issues at 0 and 1.

In [ ]:
def pobs(X):
    """
    Compute pseudo-observations from data.
    
    Pseudo-observations transform the data to the unit cube by replacing
    each value with its normalized rank. This is essential for copula fitting
    as it removes the marginal distributions.
    
    Parameters
    ----------
    X : ndarray of shape (n, d)
        Input data with n samples and d dimensions
    
    Returns
    -------
    U : ndarray of shape (n, d)
        Pseudo-observations in [0, 1]^d
    """
    n, d = X.shape
    # Divide by n+1 to avoid boundary issues (values exactly at 0 or 1)
    U = rankdata(X, method='ordinal', axis=0) / float(n + 1)
    return U

In [ ]:
# Demonstrate pseudo-observations with Gaussian copula data
# Reference: https://documentation.sas.com/doc/en/etsug/15.2/etsug_copula_details03.htm

# Generate Gaussian copula samples
r = 0.8  # Correlation parameter
P = np.asarray([
    [1, r],
    [r, 1]
])
d = P.shape[0]
n = 500

A = np.linalg.cholesky(P)
Z = np.random.normal(size=(n, d))
U_Gauss = norm.cdf(np.matmul(Z, A))

# Transform to Normal marginals
H1 = np.empty_like(U_Gauss)
H1[:, 0] = norm.ppf(U_Gauss[:, 0])
H1[:, 1] = norm.ppf(U_Gauss[:, 1])

# Transform to Beta and Cauchy marginals (different marginals, same copula)
H2 = np.empty_like(U_Gauss)
H2[:, 0] = beta.ppf(U_Gauss[:, 0], 2, 5)
H2[:, 1] = cauchy.ppf(U_Gauss[:, 1])

# Compute pseudo-observations for both cases
U1 = pobs(H1)
U2 = pobs(H2)

In [ ]:
# Visualize the data and their pseudo-observations
# Key insight: Different marginals give different joint appearances,
# but the pseudo-observations reveal the same underlying dependence structure

plt.figure(figsize=(6, 6))

# Original data with Normal marginals
plt.subplot(2, 2, 1)
plt.scatter(H1[:, 0], H1[:, 1], alpha=0.2)
tau_result = kendalltau(H1[:, 0], H1[:, 1])
plt.title(r'$\rho=%0.02f$, $\tau=%0.02f$' % (pearsonr(H1[:, 0], H1[:, 1])[0], tau_result.correlation))
plt.xlabel('X (Normal)')
plt.ylabel('Y (Normal)')

# Original data with Beta and Cauchy marginals
plt.subplot(2, 2, 2)
plt.scatter(H2[:, 0], H2[:, 1], alpha=0.2)
tau_result = kendalltau(H2[:, 0], H2[:, 1])
plt.title(r'$\rho=%0.02f$, $\tau=%0.02f$' % (pearsonr(H2[:, 0], H2[:, 1])[0], tau_result.correlation))
plt.xlabel('X (Beta)')
plt.ylabel('Y (Cauchy)')

# Pseudo-observations from Normal marginals
plt.subplot(2, 2, 3)
plt.scatter(U1[:, 0], U1[:, 1], alpha=0.2)
plt.xlabel('U')
plt.ylabel('V')
plt.title('Pseudo-observations')

# Pseudo-observations from Beta/Cauchy marginals
plt.subplot(2, 2, 4)
plt.scatter(U2[:, 0], U2[:, 1], alpha=0.2)
plt.xlabel('U')
plt.ylabel('V')
plt.title('Pseudo-observations')

plt.tight_layout()

In [ ]:
# Demonstrate pseudo-observations with Clayton copula data
# Reference: https://medium.com/@financialnoob/introduction-to-copulas-part-2-9de74010ed87

alpha = 6  # Clayton parameter
u_rand = np.random.rand(n)
t_rand = np.random.rand(n)
v_rand = ((t_rand / u_rand**(-alpha-1))**(-alpha/(1+alpha)) - u_rand**(-alpha) + 1)**(-1/alpha)
U_clayton = np.vstack([u_rand, v_rand]).T

# Transform to Beta and Cauchy marginals
H2 = np.empty_like(U_clayton)
H2[:, 0] = beta.ppf(U_clayton[:, 0], 2, 5)
H2[:, 1] = cauchy.ppf(U_clayton[:, 1])

plt.figure(figsize=(6, 3))

# Joint distribution with Beta/Cauchy marginals
plt.subplot(1, 2, 1)
plt.scatter(H2[:, 0], H2[:, 1], alpha=0.2)
plt.xlabel('$\\mathrm{Beta}(2,5)$')
plt.ylabel('$\\mathrm{Cauchy}(0,1)$')
plt.title('Original Data')

# Pseudo-observations reveal the Clayton copula structure
U = pobs(H2)
plt.subplot(1, 2, 2)
plt.scatter(U[:, 0], U[:, 1], alpha=0.2)
plt.xlabel('U')
plt.ylabel('V')
plt.title('Pseudo-observations')

plt.tight_layout()

## Parameter Estimation Using Kendall's Tau

Kendall's tau ($\tau$) is a rank-based measure of dependence that is invariant to monotonic transformations. This makes it ideal for copula parameter estimation.

### Kendall's Tau

**Theoretical definition** (in terms of copula):
$$ \tau = 4 \int_0^1 \int_0^1 C(u,v) \, dC(u,v) - 1 $$

**Sample estimator**:
$$ \hat{\tau} = \frac{2}{n(n-1)} \sum_{i<j} \text{sgn}(x_i-x_j) \cdot \text{sgn}(y_i-y_j) $$

The key advantage: Kendall's tau has simple closed-form relationships with copula parameters for many common families.

### Gaussian Copula Parameter Estimation

For the Gaussian copula with correlation parameter $\rho$:
$$ \rho = \sin\left(\tau \cdot \frac{\pi}{2}\right) $$

For Spearman's rho:
$$ \rho = 2\sin\left(\rho_S \cdot \frac{\pi}{6}\right) $$

In [ ]:
def ktau2gaussian(val, dependency='kendall'):
    """
    Convert Kendall's tau or Spearman's rho to Gaussian copula correlation parameter.
    
    Parameters
    ----------
    val : float
        Dependence measure value
    dependency : str
        Type of measure: 'kendall' or 'spearman'
    
    Returns
    -------
    rho : float
        Gaussian copula correlation parameter
    """
    if dependency == 'kendall':
        rho = np.sin(val * math.pi / 2.0)
    elif dependency == 'spearman':
        rho = 2 * np.sin(val * math.pi / 6.0)
    return rho

# Alternative: Maximum Likelihood approach (commented pseudo-code)
# fun = @(theta) -sum(log(copulapdf(family,u,theta)+eps));
# theta = fminbnd(fun,-1,1,optimset('Display','off'));

### Clayton Copula Parameter Estimation

For the Clayton copula with parameter $\alpha > 0$:
$$ \alpha = \frac{2\tau}{1-\tau} $$

Note: This relationship requires $\tau \in [0, 1)$ (positive dependence only).

In [ ]:
def ktau2clayton(val, dependency='kendall'):
    """
    Convert Kendall's tau to Clayton copula parameter.
    
    Parameters
    ----------
    val : float
        Kendall's tau value (must be in [0, 1))
    dependency : str
        Type of measure (only 'kendall' supported)
    
    Returns
    -------
    alpha : float
        Clayton copula parameter
    """
    if dependency == 'kendall':
        if val < 0 or val >= 1:
            raise ValueError("Valid values of Kendall's Tau for Clayton Copula are [0, 1)")
        alpha = 2.0 * val / (1.0 - val)
    elif dependency == 'spearman':
        raise NotImplementedError("Spearman's rho currently unsupported for Clayton Copula family!")
    
    return alpha

### Gumbel Copula Parameter Estimation

For the Gumbel copula with parameter $\alpha \geq 1$:
$$ \alpha = \frac{1}{1-\tau} $$

Note: This relationship requires $\tau \in [0, 1)$ (positive dependence only).

In [ ]:
def ktau2gumbel(val, dependency='kendall'):
    """
    Convert Kendall's tau to Gumbel copula parameter.
    
    Parameters
    ----------
    val : float
        Kendall's tau value (must be in [0, 1))
    dependency : str
        Type of measure (only 'kendall' supported)
    
    Returns
    -------
    alpha : float
        Gumbel copula parameter
    """
    if dependency == 'kendall':
        if val < 0 or val >= 1:
            raise ValueError("Valid values of Kendall's Tau for Gumbel Copula are [0, 1)")
        alpha = 1.0 / (1.0 - val)
    elif dependency == 'spearman':
        raise NotImplementedError("Spearman's rho currently unsupported for Gumbel Copula family!")
    
    return alpha

### Frank Copula Parameter Estimation

For the Frank copula, the relationship involves the Debye function $\mathcal{D}_n(x)$:

$$ \tau = 1 - \frac{4}{\alpha}\left(1 - \mathcal{D}_1(\alpha)\right) $$

where the Debye function is:
$$ \mathcal{D}_n(x) = \frac{n}{x^n} \int_0^x \frac{t^n}{e^t-1} dt $$

Since there is no closed-form inverse, we use numerical root finding.

In [ ]:
def debye(x, n):
    """
    Evaluate the Debye function.
    
    The Debye function appears in the relationship between Kendall's tau
    and the Frank copula parameter.
    
    See: http://en.wikipedia.org/wiki/Debye_function
    
    Parameters
    ----------
    x : float
        Input value
    n : int
        Order of the Debye function
    
    Returns
    -------
    float
        D_n(x)
    """
    n = float(n)  # Ensure n is a float for division
    sol = integrate.quad(lambda t: pow(t, n) / (np.exp(t) - 1.0), 0.0, x)
    return n * sol[0] / pow(x, n)


def _frank_kendall_fopt(alpha, tau):
    """Objective function for Frank copula parameter estimation."""
    return 4 * (debye(alpha, 1) - 1) / alpha + 1 - tau


def ktau2frank(val, dependency='kendall'):
    """
    Convert Kendall's tau to Frank copula parameter.
    
    Uses numerical root finding since there is no closed-form solution.
    
    Parameters
    ----------
    val : float
        Kendall's tau value
    dependency : str
        Type of measure (only 'kendall' supported)
    
    Returns
    -------
    alpha : float
        Frank copula parameter
    """
    if dependency == 'kendall':
        return fsolve(_frank_kendall_fopt, 1, args=(val))[0]
    elif dependency == 'spearman':
        raise NotImplementedError("Spearman's rho currently unsupported for Frank Copula family!")
    
    return None

In [ ]:
## Validation Examples

Let us validate our parameter estimation methods by generating data from known copulas
and checking if we can recover the true parameters.

### Gaussian Copula Validation

# True parameter
r_true = 0.5

# Generate Gaussian copula samples
P = np.asarray([
    [1, r_true],
    [r_true, 1]
])
d = P.shape[0]
n = 1000

A = np.linalg.cholesky(P)
Z = np.random.normal(size=(n, d))
U_Gauss = norm.cdf(np.matmul(Z, A))

# Transform to different marginals
H1 = np.empty_like(U_Gauss)
H1[:, 0] = norm.ppf(U_Gauss[:, 0])
H1[:, 1] = norm.ppf(U_Gauss[:, 1])

H2 = np.empty_like(U_Gauss)
H2[:, 0] = beta.ppf(U_Gauss[:, 0], 2, 5)
H2[:, 1] = cauchy.ppf(U_Gauss[:, 1])

# Estimate parameters using Kendall's tau
tau1 = kendalltau(H1[:, 0], H1[:, 1])
tau2 = kendalltau(H2[:, 0], H2[:, 1])
rho_est1 = ktau2gaussian(tau1.correlation)
rho_est2 = ktau2gaussian(tau2.correlation)

print("Gaussian Copula Parameter Recovery")
print(f"True rho: {r_true}")
print(f"Estimated rho (Normal marginals): {rho_est1:.4f}")
print(f"Estimated rho (Beta/Cauchy marginals): {rho_est2:.4f}")
print(f"Pearson correlation (Normal): {pearsonr(H1[:, 0], H1[:, 1])[0]:.4f}")
print(f"Pearson correlation (Beta/Cauchy): {pearsonr(H2[:, 0], H2[:, 1])[0]:.4f}")

In [ ]:
### Clayton Copula Validation

# True parameter
alpha_true = 6

# Generate Clayton copula samples
u_rand = np.random.rand(n)
t_rand = np.random.rand(n)
v_rand = ((t_rand / u_rand**(-alpha_true-1))**(-alpha_true/(1+alpha_true)) - u_rand**(-alpha_true) + 1)**(-1/alpha_true)
U_clayton = np.vstack([u_rand, v_rand]).T

# Transform to different marginals
H1 = np.empty_like(U_clayton)
H1[:, 0] = norm.ppf(U_clayton[:, 0])
H1[:, 1] = norm.ppf(U_clayton[:, 1])

H2 = np.empty_like(U_clayton)
H2[:, 0] = beta.ppf(U_clayton[:, 0], 2, 5)
H2[:, 1] = cauchy.ppf(U_clayton[:, 1])

# Estimate parameters using Kendall's tau
tau1 = kendalltau(H1[:, 0], H1[:, 1])
tau2 = kendalltau(H2[:, 0], H2[:, 1])
alpha_est1 = ktau2clayton(tau1.correlation)
alpha_est2 = ktau2clayton(tau2.correlation)

print("\nClayton Copula Parameter Recovery")
print(f"True alpha: {alpha_true}")
print(f"Estimated alpha (Normal marginals): {alpha_est1:.4f}")
print(f"Estimated alpha (Beta/Cauchy marginals): {alpha_est2:.4f}")

In [ ]:
### Gumbel Copula Validation

# Gumbel copula generator functions
def gumbel_phi(t, alpha):
    """Gumbel generator function"""
    return (-np.log(t))**alpha

def gumbel_phi_inv(t, alpha):
    """Inverse of Gumbel generator function"""
    return np.exp(-t**(1/alpha))

def gumbel_K(t, alpha):
    """Kendall distribution function for Gumbel copula"""
    return t * (alpha - np.log(t)) / alpha

# True parameter
alpha_true = 6

# Generate Gumbel copula samples
t1_rand = np.random.rand(n)
t2_rand = np.random.rand(n)

w = []
for t_val in t2_rand:
    func = lambda w: gumbel_K(w, alpha=alpha_true) - t_val
    w.append(brentq(func, 0.0000000001, 0.9999999999))
w = np.array(w).flatten()

u = gumbel_phi_inv(t1_rand * gumbel_phi(w, alpha=alpha_true), alpha=alpha_true)
v = gumbel_phi_inv((1-t1_rand) * gumbel_phi(w, alpha=alpha_true), alpha=alpha_true)
U_gumbel = np.vstack([u, v]).T

# Transform to different marginals
H1 = np.empty_like(U_gumbel)
H1[:, 0] = norm.ppf(U_gumbel[:, 0])
H1[:, 1] = norm.ppf(U_gumbel[:, 1])

H2 = np.empty_like(U_gumbel)
H2[:, 0] = beta.ppf(U_gumbel[:, 0], 2, 5)
H2[:, 1] = cauchy.ppf(U_gumbel[:, 1])

# Estimate parameters using Kendall's tau
tau1 = kendalltau(H1[:, 0], H1[:, 1])
tau2 = kendalltau(H2[:, 0], H2[:, 1])
alpha_est1 = ktau2gumbel(tau1.correlation)
alpha_est2 = ktau2gumbel(tau2.correlation)

print("\nGumbel Copula Parameter Recovery")
print(f"True alpha: {alpha_true}")
print(f"Estimated alpha (Normal marginals): {alpha_est1:.4f}")
print(f"Estimated alpha (Beta/Cauchy marginals): {alpha_est2:.4f}")

In [ ]:
### Frank Copula Validation

# Frank copula generator functions
def frank_phi(t, alpha):
    """Frank generator function"""
    return -np.log((np.exp(-alpha*t) - 1) / (np.exp(-alpha) - 1))

def frank_phi_inv(t, alpha):
    """Inverse of Frank generator function"""
    return -1/alpha * np.log((np.exp(-alpha) - 1) / np.exp(t) + 1)

def frank_K(t, alpha):
    """Kendall distribution function for Frank copula"""
    return (t + (1 - np.exp(alpha*t)) * np.log((1-np.exp(alpha*t)) * 
                                               np.exp(-alpha*t+alpha) / (1-np.exp(alpha))) / alpha)

# True parameter
alpha_true = 6

# Generate Frank copula samples
t1_rand = np.random.rand(n)
t2_rand = np.random.rand(n)

w = []
for t_val in t2_rand:
    func = lambda w: frank_K(w, alpha=alpha_true) - t_val
    w.append(brentq(func, 0.0000000001, 0.9999999999))
w = np.array(w).flatten()

u = frank_phi_inv(t1_rand * frank_phi(w, alpha=alpha_true), alpha=alpha_true)
v = frank_phi_inv((1-t1_rand) * frank_phi(w, alpha=alpha_true), alpha=alpha_true)
U_frank = np.vstack([u, v]).T

# Transform to different marginals
H1 = np.empty_like(U_frank)
H1[:, 0] = norm.ppf(U_frank[:, 0])
H1[:, 1] = norm.ppf(U_frank[:, 1])

H2 = np.empty_like(U_frank)
H2[:, 0] = beta.ppf(U_frank[:, 0], 2, 5)
H2[:, 1] = cauchy.ppf(U_frank[:, 1])

# Estimate parameters using Kendall's tau
tau1 = kendalltau(H1[:, 0], H1[:, 1])
tau2 = kendalltau(H2[:, 0], H2[:, 1])
alpha_est1 = ktau2frank(tau1.correlation)
alpha_est2 = ktau2frank(tau2.correlation)

print("\nFrank Copula Parameter Recovery")
print(f"True alpha: {alpha_true}")
print(f"Estimated alpha (Normal marginals): {alpha_est1:.4f}")
print(f"Estimated alpha (Beta/Cauchy marginals): {alpha_est2:.4f}")